In [80]:
import os
import zipfile
%load_ext autoreload

%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [94]:
import tracemalloc
import psutil
import pprint
import pandas as pd
import uuid
import json
import os
import glob
import re
import sys
from bs4 import NavigableString, BeautifulSoup
from collections import defaultdict
import random
import string
import time

from utils.config import config
from utils.logger.logger import loggerCreator

# ePI Modules
from parse.rulebook.rulebook import StyleRulesDictionary

from parse.extractor.parser import parserExtractor
from match.matchDocument.matchDocument import MatchDocument
from documentAnnotation.documentAnnotation import DocumentAnnotation
from htmlDocTypePartitioner.partition import DocTypePartitioner
from extractContentBetweenHeadings.dataBetweenHeadingsExtractor import DataBetweenHeadingsExtractor
from fhirXmlGenerator.fhirXmlGenerator import FhirXmlGenerator
from fhirService.fhirService import FhirService
from utils.logger.matchLogger import MatchLogger
from languageInfo.documentTypeNames.documentTypeNames import DocumentTypeNames
from listBundle.addAndUpdateListBundle.addAndUpdateListBundle import ListBundleHandler

class FolderNotFoundError(Exception):
    pass

class Metrics:
    
    def __init__(self, logFileName, logger):
        self.logFileName = logFileName
        self.start()
        self.writer = open(self.logFileName, 'a')
        self.writer.write("StepName,Time,Current Memory,Peak Memory,Used Ram Percentage\n")
        self.finalPeak = 0
        self.finalTotalTime = 0
        self.finalUsedRamPerc = 0
        self.logger = logger
    
    def start(self):
        self.startTime = time.time()
        tracemalloc.start()
    
    def getMetric(self, msg):
        
        self.endTime = time.time()
        
        self.totalTime = self.endTime - self.startTime
        
        
        current, peak = tracemalloc.get_traced_memory()
        current = current / 10**6
        peak = peak / 10**6
        
        usedRamPerc = psutil.virtual_memory()[2]
        
        self.finalPeak = max(self.finalPeak, peak)
        self.finalUsedRamPerc = max(self.finalUsedRamPerc, usedRamPerc)

        self.finalTotalTime = self.finalTotalTime + self.totalTime
        #self.finalTotalTime = round(self.finalTotalTime/60,3)
        
        outputString = f"{msg},{round(self.totalTime/60,4)} Min,{current} MB,{peak} MB,{usedRamPerc}%\n"
        
        self.logger.logFlowCheckpoint(f"{outputString}")
        
        print(f"Metrics : {outputString}")
        self.writer.write(outputString)
        tracemalloc.stop()
        tracemalloc.start()
        self.startTime = time.time()
    def end(self):
        
        current, peak = tracemalloc.get_traced_memory()
        current = current / 10**6
        outputString = f"Final Metrics,{round(self.finalTotalTime/60,4)} Min,{current} MB,{self.finalPeak} MB,{self.finalUsedRamPerc}%\n"
        print(f"Metrics : {outputString}")
        self.logger.logFlowCheckpoint(f"{outputString}")
        self.writer.write(outputString)
        self.writer.close()
        tracemalloc.stop()
        
        


def convertToInt(x):
    try:
        return str(int(x))
    except:
        return x


def convertCollectionToDataFrame(collection):

    dfExtractedHier = pd.DataFrame(collection)
    dfExtractedHier['parent_id'] = dfExtractedHier['parent_id'].apply(
        lambda x: convertToInt(x))
    dfExtractedHier['id'] = dfExtractedHier['id'].apply(
        lambda x: convertToInt(x))

    return dfExtractedHier

def getRandomString(N):
    str_ = ''.join(random.choice(string.ascii_uppercase + string.digits
                                 + string.ascii_lowercase) for _ in range(N))
    return str_


def convertHtmlToJson(controlBasePath, basePath, domain, procedureType, languageCode, htmlDocName, fileNameQrd, fileNameLog):

    module_path = os.path.join(basePath)

    if "/" in basePath:
        pathSep = "/"
    else:
        pathSep = "\\"
    
    # Generate output folder path
    output_json_path = os.path.join(basePath, 'outputJSON')

    """
        Check if input folder exists, else throw exception
    """
    if(os.path.exists(module_path)):
        filenames = glob.glob(os.path.join(module_path, htmlDocName))

        # Create language specific folder in outputJSON folder if it doesn't exist
        if(not os.path.exists(output_json_path)):
            os.mkdir(output_json_path)
        logger = MatchLogger(f'Parser_{getRandomString(1)}', htmlDocName,
                             domain, procedureType, languageCode, "HTML", fileNameLog)

        styleLogger = MatchLogger(
            f'Style Dictionary_{getRandomString(1)}', htmlDocName, domain, procedureType, languageCode, "HTML", fileNameLog)

        styleRulesObj = StyleRulesDictionary(logger=styleLogger,
                                             controlBasePath=controlBasePath,
                                             language=languageCode,
                                             fileName=fileNameQrd,
                                             domain=domain,
                                             procedureType=procedureType
                                             )

        parserObj = parserExtractor(config, logger, styleRulesObj.styleRuleDict,
                                    styleRulesObj.styleFeatureKeyList,
                                    styleRulesObj.qrd_section_headings)
        
        for input_filename in filenames:
          # if(input_filename.find('Kalydeco II-86-PI-clean')!=-1):
            output_filename = os.path.join(output_json_path, htmlDocName)
            style_filepath =  output_filename.replace('.html','.txt')
            style_filepath =  style_filepath.replace('.txtl','.txt')
            style_filepath =  style_filepath.replace('.htm','.txt')
            print("-------------",style_filepath,"-----------------")

            output_filename = output_filename.replace('.html', '.json')
            output_filename = output_filename.replace('.htm', '.json')
            print(input_filename, output_filename)
            parserObj.createPIJsonFromHTML(input_filepath=input_filename,
                                           output_filepath=output_filename,
                                           style_filepath = style_filepath,
                                           img_base64_dict=parserObj.convertImgToBase64(input_filename)
                                           )
        #return parserObj, input_filename, output_filename, style_filepath
        return output_filename.split(pathSep)[-1], style_filepath
    else:
        try:    
            raise FolderNotFoundError(module_path + " not found")
        except:  
            logger.logFlowCheckpoint("Folder For Language Code Not Found In Input File")
            logger.logException("Folder For Language Code Not Found In Input File")
        raise FolderNotFoundError(module_path + " not found")
        return None


def splitJson(controlBasePath, basePath, domain, procedureType, languageCode, fileNameJson, fileNameQrd, fileNameLog):

    styleLogger = MatchLogger(
        f'Style Dictionary_{getRandomString(1)}', fileNameJson, domain, procedureType, languageCode, "Json", fileNameLog)

    styleRulesObj = StyleRulesDictionary(logger=styleLogger,
                                        controlBasePath=controlBasePath,
                                        language=languageCode,
                                        fileName=fileNameQrd,
                                        domain=domain,
                                        procedureType=procedureType
                                        )
    
    path_json = os.path.join(basePath,'outputJSON', fileNameJson)
    print("PathJson",path_json)
    partitionLogger = MatchLogger(
        f'Partition_{getRandomString(1)}', fileNameJson, domain, procedureType, languageCode, "Json", fileNameLog)

    partitioner = DocTypePartitioner(partitionLogger, domain, procedureType)

    partitionedJsonPaths = partitioner.partitionHtmls(
        styleRulesObj.qrd_section_headings, path_json)

    return partitionedJsonPaths


def extractAndValidateHeadings(controlBasePath,
                                basePath,
                                domain,
                                procedureType,
                                languageCode,
                                documentNumber,
                                fileNameDoc,
                                fileNameQrd,
                                fileNameMatchRuleBook,
                                fileNameDocumentTypeNames,
                                fileNameLog,
                                stopWordFilterLen=6,
                                isPackageLeaflet=False,
                                medName=None
                                ):

    if documentNumber == 0:
        topHeadingsConsidered = 4
        bottomHeadingsConsidered = 6
    elif documentNumber == 1:
        topHeadingsConsidered = 3
        bottomHeadingsConsidered = 5
    elif documentNumber == 2:
        topHeadingsConsidered = 5
        bottomHeadingsConsidered = 15
    else:
        topHeadingsConsidered = 5
        bottomHeadingsConsidered = 10

    print(f"Starting Heading Extraction For File :- {fileNameDoc}")
    logger = MatchLogger(f"Heading Extraction {fileNameDoc}_{getRandomString(1)}", fileNameDoc, domain, procedureType, languageCode, documentNumber, fileNameLog)
    logger.logFlowCheckpoint("Starting Heading Extraction")

    stopWordlanguage = DocumentTypeNames(
        controlBasePath=controlBasePath,
        fileNameDocumentTypeNames=fileNameDocumentTypeNames,
        languageCode=languageCode,
        domain=domain,
        procedureType=procedureType,
        documentNumber=documentNumber
        ).extractStopWordLanguage()

    matchDocObj = MatchDocument(
        logger,
        controlBasePath,
        basePath,
        domain,
        procedureType,
        languageCode,
        documentNumber,
        fileNameDoc,
        fileNameQrd,
        fileNameMatchRuleBook,
        fileNameDocumentTypeNames,
        topHeadingsConsidered,
        bottomHeadingsConsidered,
        stopWordFilterLen,
        stopWordlanguage,
        isPackageLeaflet,
        medName)
    df, coll, documentType = matchDocObj.matchHtmlHeaddingsWithQrd()

    return df, coll, documentType


def parseDocument(controlBasePath,
                  basePath,
                  htmlDocName,
                  fileNameQrd,
                  fileNameMatchRuleBook,
                  fileNameDocumentTypeNames,
                  jsonTempFileName,
                  listBundleDocumentTypeCodesFileName,
                  apiMmgtBaseUrl,
                  getListApiEndPointUrlSuffix,
                  addUpdateListApiEndPointUrlSuffix,
                  addBundleApiEndPointUrlSuffix,
                  apiMmgtSubsKey,
                  medName = None):
    
    listRegulatedAuthCodesAccrossePI = []
    
    if "/" in basePath:
        pathSep = "/"        
    else:
        pathSep = "\\"
    
    fileNameLog = os.path.join(basePath,'FinalLog.txt')

    pathComponents = basePath.split(pathSep)
    print(pathComponents, htmlDocName)
    timestamp = pathComponents[-1]
    languageCode =  pathComponents[-2]
    medName = pathComponents[-3]
    procedureType = pathComponents[-4]
    domain = pathComponents[-5]

    print(timestamp, languageCode, medName, procedureType, domain)
        
    flowLogger =  MatchLogger(f"Flow Logger HTML_{getRandomString(1)}", htmlDocName, domain, procedureType, languageCode, "HTML", fileNameLog)
    
    metrics = Metrics(os.path.join(basePath,'Metrics.csv'),flowLogger)
    
    
    flowLogger.logFlowCheckpoint("Starting HTML Conversion To Json")
    ###Convert Html to Json
    
    fileNameJson, stylesFilePath = convertHtmlToJson(controlBasePath, basePath, domain, procedureType, languageCode, htmlDocName, fileNameQrd, fileNameLog)
    
    print("stylePath:-",stylesFilePath)
    flowLogger.logFlowCheckpoint("Completed HTML Conversion To Json")
    metrics.getMetric("HTML Conversion To Json")

    flowLogger.logFlowCheckpoint("Starting Json Split")

    ###Split Uber Json to multiple Jsons for each category.
    partitionedJsonPaths = splitJson(controlBasePath, basePath, domain, procedureType, languageCode, fileNameJson, fileNameQrd, fileNameLog)
    
    partitionedJsonPaths = [ path.split(pathSep)[-1] for path in partitionedJsonPaths]
    flowLogger.logFlowCheckpoint(str(partitionedJsonPaths))
    
    flowLogger.logFlowCheckpoint("Completed Json Split")
    metrics.getMetric("Split Json")
    
    flowLogger.logFlowCheckpoint("Started Processing Partitioned Jsons")
    
    for index, fileNamePartitioned in enumerate(partitionedJsonPaths):
        print("Index", index)
        #if index in [0,2,3]:
        #    continue
        
        flowLogger.logFlowCheckpoint(f"\n\n\n\n||||||||||||||||||||||||||||||||{str(index)} ||||| {str(fileNamePartitioned)}||||||||||||||||||||||||||||||||\n\n\n\n")
        
        if index == 3:
            stopWordFilterLen = 100
            isPackageLeaflet = True
        else:
            stopWordFilterLen = 6
            isPackageLeaflet = False
            
        df, coll, documentType = extractAndValidateHeadings(controlBasePath,
                                    basePath,
                                    domain,
                                    procedureType,
                                    languageCode,
                                    index,
                                    fileNamePartitioned,
                                    fileNameQrd,
                                    fileNameMatchRuleBook,
                                    fileNameDocumentTypeNames,
                                    fileNameLog,
                                    stopWordFilterLen=stopWordFilterLen,
                                    isPackageLeaflet=isPackageLeaflet,
                                    medName=medName)
        #return df, coll, documentType
        print(f"Completed Heading Extraction For File")
        flowLogger.logFlowCheckpoint("Completed Heading Extraction For File")
        metrics.getMetric(f"{index}: Heading Extraction")

        print(f"Starting Document Annotation For File :- {fileNamePartitioned}")        
        flowLogger.logFlowCheckpoint("Starting Document Annotation For File")
        documentAnnotationObj = DocumentAnnotation(fileNamePartitioned,'c20835db4b1b4e108828a8537ff41506','https://spor-sit.azure-api.net/pms/api/v2/',df,coll, index)
        try:
            pms_oms_annotation_data = documentAnnotationObj.processRegulatedAuthorizationForDoc()
            print(pms_oms_annotation_data)
        except Exception as e:
            pms_oms_annotation_data = None
            print("Error Found", str(e))
            
        print(f"Completed Document Annotation")        
        flowLogger.logFlowCheckpoint("Completed Document Annotation")
        metrics.getMetric(f"{index}: Document Annotation")
        
        print(f"Starting Extracting Content Between Heading For File :- {fileNamePartitioned}")        
        flowLogger.logFlowCheckpoint("Starting Extracting Content Between Heading")
        
        extractContentlogger =  MatchLogger(f'ExtractContentBetween_{index}_{getRandomString(1)}', fileNamePartitioned, domain, procedureType, languageCode, index, fileNameLog)
        extractorObj = DataBetweenHeadingsExtractor(extractContentlogger, basePath, coll)
        dfExtractedHierRR = extractorObj.extractContentBetweenHeadings(fileNamePartitioned)
        
        print(f"Completed Extracting Content Between Heading")        
        flowLogger.logFlowCheckpoint("Completed Extracting Content Between Heading")
        metrics.getMetric(f"{index}: Content Extraction")
        
        
        xmlLogger =  MatchLogger(f'XmlGeneration_{index}_{getRandomString(1)}', fileNamePartitioned, domain, procedureType, languageCode, index, fileNameLog)
        
        listBundleDocumentTypeCodesFilePath = os.path.join(controlBasePath,
                                                                listBundleDocumentTypeCodesFileName.split(".")[0],
                                                                listBundleDocumentTypeCodesFileName)
        with open(listBundleDocumentTypeCodesFilePath, encoding='utf-8') as f:
            listBundleDocumentTypeCodes = json.load(f)
            
        bundleDocumentTypeCode = listBundleDocumentTypeCodes[domain][str(index)]['listBundleCode']
        bundleMetaData = {'pmsOmsAnnotationData':pms_oms_annotation_data,
                          'documentTypeCode': bundleDocumentTypeCode,
                          'documentType': documentType,
                          'medName': medName}
        
        fhirXmlGeneratorObj = FhirXmlGenerator(xmlLogger, controlBasePath, basePath, bundleMetaData, stylesFilePath)
        fileNameXml = fileNamePartitioned.replace('.json','.xml')
        generatedXml = fhirXmlGeneratorObj.generateXml(dfExtractedHierRR, fileNameXml)
        
        metrics.getMetric(f"{index}: Generate XML")
        
        fhirServiceLogger =  MatchLogger(f'XML Submission Logger_{index}_{getRandomString(1)}', fileNamePartitioned, domain, procedureType, languageCode, index, fileNameLog)
        
        fhirServiceObj = FhirService(fhirServiceLogger, apiMmgtBaseUrl, addBundleApiEndPointUrlSuffix, apiMmgtSubsKey, basePath, generatedXml)
        fhirServiceObj.submitFhirXml()
        
        
        
        
        metrics.getMetric(f"{index}: Submit FHIR Msg")
        
        print(f"Created XML File For :- {fileNamePartitioned}")
        
        flowLogger.logFlowCheckpoint("Starting list bundle update/addition")
        if documentAnnotationObj.listRegulatedAuthorizationIdentifiers != None:
            for id in documentAnnotationObj.listRegulatedAuthorizationIdentifiers:
                listRegulatedAuthCodesAccrossePI.append(id)
        listBundleLogger =  MatchLogger(f'List Bundle Creation Logger_{index}_{getRandomString(1)}', fileNamePartitioned, domain, procedureType, languageCode, index, fileNameLog)
        print("\nlistRegulatedAuthCodesAccrossePI",listRegulatedAuthCodesAccrossePI)
        try:
            listBundleHandler = ListBundleHandler(listBundleLogger,
                     domain,
                     procedureType,
                     index,
                     documentType,
                     languageCode,
                     medName,
                     controlBasePath,
                     jsonTempFileName,
                     listBundleDocumentTypeCodesFileName,
                     fileNameDocumentTypeNames,
                     listRegulatedAuthCodesAccrossePI,
                     apiMmgtBaseUrl,
                     getListApiEndPointUrlSuffix,
                     addUpdateListApiEndPointUrlSuffix,
                     apiMmgtSubsKey)

            listBundleXml = listBundleHandler.addOrUpdateDocumentItem(str(fhirServiceObj.SubmittedFhirMsgRefId))
            print(listBundleXml)
            listBundleHandler.submitListXmLToServer(listBundleXml)

            flowLogger.logFlowCheckpoint("Completed list bundle update/addition")
            metrics.getMetric(f"{index}: Update/Add List Bundle")
            #return df,coll,dfExtractedHierRR
        except Exception as e:
            print(str(e))
            if 'No MAN Code found' in str(e):
                flowLogger.logFlowCheckpoint("Skipping list bundle addtion/update as no MAN found")
            
    
    flowLogger.logFlowCheckpoint("Completed Processing Partitioned Jsons")
    metrics.getMetric(f"{index}: Completed")
    metrics.end()

In [95]:
#from wordToHtmlConvertor.wordToHtmlConvertor import WordToHtmlConvertor
#
#wordToHtmlConvertorObj = WordToHtmlConvertor()
#wordToHtmlConvertorObj.convertWordToHTML()

In [96]:
os.path.abspath(os.path.join('..'))

'F:\\Projects\\EMA\\Repository\\EMA EPI PoC\\function_code'

In [97]:
def runAll(inputList):
    for inputDoc in inputList:
        # inputZipFolderPath = "F:\Projects\EMA\Repository\EMA EPI PoC\\function_code\\inputblob"
        inputZipFolderPath = os.path.abspath(os.path.join('..'))
        inputZipFolderPath = os.path.join(inputZipFolderPath, 'inputblob')
        inputZipFileName = inputDoc

        fileNameQrd = 'qrd_canonical_model.csv'
        fileNameMatchRuleBook = 'ruleDict.json'
        fileNameDocumentTypeNames = 'documentTypeNames.json'
        fsMountName = '/mounted'
        jsonTempFileName = 'listBundleJsonTemplate.json'
        listBundleDocumentTypeCodesFileName = 'listBundleDocumentTypeCodes.json'
        apiMmgtBaseUrl = "https://ema-dap-epi-dev-fhir-apim.azure-api.net"
        getListApiEndPointUrlSuffix = "/epi/v1/List"
        addUpdateListApiEndPointUrlSuffix = "/epi-w/v1/List"
        addBundleApiEndPointUrlSuffix = "/epi-w/v1/Bundle"
        apiMmgtSubsKey = "ba6d7e9a73ed4facaa58fc983bf6db50"



        info = inputZipFileName.split("~")

        try:
            medName = info[0]
            domain = info[1]
            procedureType = info[2]
            languageCode = info[3]
            timestamp = info[4]
            timestamp = timestamp.replace(".zip","")

        except Exception:
            raise f"Missing required info in the zip file name {inputZipFileName}"

        if "\\" in os.getcwd():
            localEnv = True
            inputZipFolderPath = os.path.join(os.path.abspath(os.path.join('..')),inputZipFolderPath)
            outputFolderPath = os.path.join(os.path.abspath(os.path.join('..')), 'work', f"{domain}", f"{procedureType}", f"{medName}", f"{languageCode}", f"{timestamp}")
            controlFolderPath = os.path.join(os.path.abspath(os.path.join('..')),'control')
        else:
            localEnv = False
            inputZipFolderPath = os.path.join(f'{fsMountName}',inputZipFolderPath)
            outputFolderPath = os.path.join(f'{fsMountName}', 'work', f"{domain}", f"{procedureType}", f"{medName}", f"{languageCode}", f"{timestamp}")
            controlFolderPath = os.path.join(f'{fsMountName}','control')


        print(inputZipFileName, inputZipFolderPath, outputFolderPath, controlFolderPath)

        mode = 0o666

        if localEnv is True:
            inputZipFolderPath = inputZipFolderPath.replace("/","\\")
            outputFolderPath = outputFolderPath.replace("/","\\")
            controlFolderPath = controlFolderPath.replace("/","\\")

        try:
            os.makedirs(inputZipFolderPath, mode)
            os.makedirs(outputFolderPath, mode)
            os.makedirs(controlFolderPath, mode)

        except Exception:
            print("Already Present")

        with zipfile.ZipFile(f'{inputZipFolderPath}/{inputZipFileName}',"r") as zip_ref:
                zip_ref.extractall(outputFolderPath)


        _,_,fileNames = next(os.walk(outputFolderPath))
        htmlFileName = [fileName for fileName in fileNames if ".htm" in fileName][0]

        print(htmlFileName)

        parseDocument(controlFolderPath,
                  outputFolderPath,
                  htmlFileName,
                  fileNameQrd,
                  fileNameMatchRuleBook,
                  fileNameDocumentTypeNames,
                  jsonTempFileName,
                  listBundleDocumentTypeCodesFileName,
                  apiMmgtBaseUrl,
                  getListApiEndPointUrlSuffix,
                  addUpdateListApiEndPointUrlSuffix,
                  addBundleApiEndPointUrlSuffix,
                  apiMmgtSubsKey,
                  medName)


In [101]:
def runAllTest(inputList):
    for inputDoc in inputList:
        # inputZipFolderPath = "F:\Projects\EMA\Repository\EMA EPI PoC\\function_code\\inputblob"
        inputZipFolderPath = os.path.abspath(os.path.join('..'))
        inputZipFolderPath = os.path.join(inputZipFolderPath, 'inputblob')
        inputZipFileName = inputDoc
        
        fileNameQrd = 'qrd_canonical_model.csv'
        fileNameMatchRuleBook = 'ruleDict.json'
        fileNameDocumentTypeNames = 'documentTypeNames.json'
        fsMountName = '/mounted'
        jsonTempFileName = 'listBundleJsonTemplate.json'
        listBundleDocumentTypeCodesFileName = 'listBundleDocumentTypeCodes.json'
        apiMmgtBaseUrl = "https://ema-dap-epi-tst-fhir-apim.azure-api.net"
        getListApiEndPointUrlSuffix = "/epi/v1/List"
        addUpdateListApiEndPointUrlSuffix = "/epi-w/v1/List"
        addBundleApiEndPointUrlSuffix = "/epi-w/v1/Bundle"
        apiMmgtSubsKey = "9e9d47b8a08148f9833e17462d90574a"




        info = inputZipFileName.split("~")

        try:
            medName = info[0]
            domain = info[1]
            procedureType = info[2]
            languageCode = info[3]
            timestamp = info[4]
            timestamp = timestamp.replace(".zip","")

        except Exception:
            raise f"Missing required info in the zip file name {inputZipFileName}"

        if "\\" in os.getcwd():
            localEnv = True
            inputZipFolderPath = os.path.join(os.path.abspath(os.path.join('..')),inputZipFolderPath)
            outputFolderPath = os.path.join(os.path.abspath(os.path.join('..')), 'work', f"{domain}", f"{procedureType}", f"{medName}", f"{languageCode}", f"{timestamp}")
            controlFolderPath = os.path.join(os.path.abspath(os.path.join('..')),'control')
        else:
            localEnv = False
            inputZipFolderPath = os.path.join(f'{fsMountName}',inputZipFolderPath)
            outputFolderPath = os.path.join(f'{fsMountName}', 'work', f"{domain}", f"{procedureType}", f"{medName}", f"{languageCode}", f"{timestamp}")
            controlFolderPath = os.path.join(f'{fsMountName}','control')


        print(inputZipFileName, inputZipFolderPath, outputFolderPath, controlFolderPath)

        mode = 0o666

        if localEnv is True:
            inputZipFolderPath = inputZipFolderPath.replace("/","\\")
            outputFolderPath = outputFolderPath.replace("/","\\")
            controlFolderPath = controlFolderPath.replace("/","\\")

        try:
            os.makedirs(inputZipFolderPath, mode)
            os.makedirs(outputFolderPath, mode)
            os.makedirs(controlFolderPath, mode)

        except Exception:
            print("Already Present")

        with zipfile.ZipFile(f'{inputZipFolderPath}/{inputZipFileName}',"r") as zip_ref:
                zip_ref.extractall(outputFolderPath)


        _,_,fileNames = next(os.walk(outputFolderPath))
        htmlFileName = [fileName for fileName in fileNames if ".htm" in fileName][0]

        print(htmlFileName)

        parseDocument(controlFolderPath,
                  outputFolderPath,
                  htmlFileName,
                  fileNameQrd,
                  fileNameMatchRuleBook,
                  fileNameDocumentTypeNames,
                  jsonTempFileName,
                  listBundleDocumentTypeCodesFileName,
                  apiMmgtBaseUrl,
                  getListApiEndPointUrlSuffix,
                  addUpdateListApiEndPointUrlSuffix,
                  addBundleApiEndPointUrlSuffix,
                  apiMmgtSubsKey,
                  medName)


In [99]:
############# EL

In [100]:
inputList = ['Elocta~H~CAP~el~2021-06-07T06-03-45Z.zip']

runAll(inputList)

2021-06-19 00:26:13,724 : Flow Logger HTML_o : Starting HTML Conversion To Json | H | CAP |  el | HTML | Elocta_clean.htm
2021-06-19 00:26:13,735 : Style Dictionary_j : Reading style dictionary in file: rule_dictionary_el.json | H | CAP |  el | HTML | Elocta_clean.htm
2021-06-19 00:26:13,805 : Style Dictionary_j : Qrd Section Keys Retrieved For Style Dictionary: ΠΕΡΙΛΗΨΗ ΤΩΝ ΧΑΡΑΚΤΗΡΙΣΤΙΚΩΝ ΤΟΥ ΠΡΟΪΟΝΤΟΣ, ΠΑΡΑΡΤΗΜΑ II, A. ΕΠΙΣΗΜΑΝΣΗ, B. ΦΥΛΛΟ ΟΔΗΓΙΩΝ ΧΡΗΣΗΣ | H | CAP |  el | HTML | Elocta_clean.htm


Elocta~H~CAP~el~2021-06-07T06-03-45Z.zip F:\Projects\EMA\Repository\EMA EPI PoC\function_code\inputblob F:\Projects\EMA\Repository\EMA EPI PoC\function_code\work\H\CAP\Elocta\el\2021-06-07T06-03-45Z F:\Projects\EMA\Repository\EMA EPI PoC\function_code\control
Already Present
Elocta_clean.htm
['F:', 'Projects', 'EMA', 'Repository', 'EMA EPI PoC', 'function_code', 'work', 'H', 'CAP', 'Elocta', 'el', '2021-06-07T06-03-45Z'] Elocta_clean.htm
2021-06-07T06-03-45Z el Elocta CAP H
------------- F:\Projects\EMA\Repository\EMA EPI PoC\function_code\work\H\CAP\Elocta\el\2021-06-07T06-03-45Z\outputJSON\Elocta_clean.txt -----------------
F:\Projects\EMA\Repository\EMA EPI PoC\function_code\work\H\CAP\Elocta\el\2021-06-07T06-03-45Z\Elocta_clean.htm F:\Projects\EMA\Repository\EMA EPI PoC\function_code\work\H\CAP\Elocta\el\2021-06-07T06-03-45Z\outputJSON\Elocta_clean.json


2021-06-19 00:26:16,945 : Parser_T : Style Information Stored In File: F:\Projects\EMA\Repository\EMA EPI PoC\function_code\work\H\CAP\Elocta\el\2021-06-07T06-03-45Z\outputJSON\Elocta_clean.txt | H | CAP |  el | HTML | Elocta_clean.htm


!!!!!!!!!!! IN TABLE !!!!!!!!!!!!!!!!!
!!!!!!!!!!! IN TABLE !!!!!!!!!!!!!!!!!
!!!!!!!!!!! IN TABLE !!!!!!!!!!!!!!!!!
!!!!!!!!!!! IN TABLE !!!!!!!!!!!!!!!!!
!!!!!!!!!!! IN TABLE !!!!!!!!!!!!!!!!!
!!!!!!!!!!! IN TABLE !!!!!!!!!!!!!!!!!


2021-06-19 00:26:30,267 : Parser_T : Writing to file: F:\Projects\EMA\Repository\EMA EPI PoC\function_code\work\H\CAP\Elocta\el\2021-06-07T06-03-45Z\outputJSON\Elocta_clean.json | H | CAP |  el | HTML | Elocta_clean.htm
2021-06-19 00:26:31,085 : Flow Logger HTML_o : Completed HTML Conversion To Json | H | CAP |  el | HTML | Elocta_clean.htm
2021-06-19 00:26:31,087 : Flow Logger HTML_o : HTML Conversion To Json,0.2894 Min,8.24692 MB,25.819801 MB,54.1%
 | H | CAP |  el | HTML | Elocta_clean.htm
2021-06-19 00:26:31,103 : Flow Logger HTML_o : Starting Json Split | H | CAP |  el | HTML | Elocta_clean.htm
2021-06-19 00:26:31,110 : Style Dictionary_O : Reading style dictionary in file: rule_dictionary_el.json | H | CAP |  el | Json | Elocta_clean.json
2021-06-19 00:26:31,172 : Style Dictionary_O : Qrd Section Keys Retrieved For Style Dictionary: ΠΕΡΙΛΗΨΗ ΤΩΝ ΧΑΡΑΚΤΗΡΙΣΤΙΚΩΝ ΤΟΥ ΠΡΟΪΟΝΤΟΣ, ΠΑΡΑΡΤΗΜΑ II, A. ΕΠΙΣΗΜΑΝΣΗ, B. ΦΥΛΛΟ ΟΔΗΓΙΩΝ ΧΡΗΣΗΣ | H | CAP |  el | Json | Elocta_clean.json
2021-06-1

...

POST sucessful: XML added with id 025dac3d-dda7-4852-9127-aa076c1e037a
Metrics : 3: Submit FHIR Msg,0.0883 Min,0.243237 MB,3.761202 MB,54.1%

Created XML File For :- Elocta_clean_ ΦΥΛΛΟ ΟΔΗΓΙΩΝ ΧΡΗΣΗΣ.json

listRegulatedAuthCodesAccrossePI ['EU/1/15/1046/001', 'EU/1/15/1046/002', 'EU/1/15/1046/003', 'EU/1/15/1046/004', 'EU/1/15/1046/005', 'EU/1/15/1046/006', 'EU/1/15/1046/007', 'EU/1/15/1046/008', 'EU/1/15/1046/001', 'EU/1/15/1046/002', 'EU/1/15/1046/003', 'EU/1/15/1046/004', 'EU/1/15/1046/005', 'EU/1/15/1046/006', 'EU/1/15/1046/007', 'EU/1/15/1046/008']


2021-06-19 00:39:11,290 : List Bundle Creation Logger_3_m : Getting list bundle for MAN EU/1/15/1046/002  | H | CAP |  el | 3 | Elocta_clean_ ΦΥΛΛΟ ΟΔΗΓΙΩΝ ΧΡΗΣΗΣ.json
2021-06-19 00:39:12,180 : List Bundle Creation Logger_3_m : Getting list bundle for MAN EU/1/15/1046/003  | H | CAP |  el | 3 | Elocta_clean_ ΦΥΛΛΟ ΟΔΗΓΙΩΝ ΧΡΗΣΗΣ.json
2021-06-19 00:39:13,040 : List Bundle Creation Logger_3_m : Getting list bundle for MAN EU/1/15/1046/004  | H | CAP |  el | 3 | Elocta_clean_ ΦΥΛΛΟ ΟΔΗΓΙΩΝ ΧΡΗΣΗΣ.json
2021-06-19 00:39:13,885 : List Bundle Creation Logger_3_m : Getting list bundle for MAN EU/1/15/1046/005  | H | CAP |  el | 3 | Elocta_clean_ ΦΥΛΛΟ ΟΔΗΓΙΩΝ ΧΡΗΣΗΣ.json
2021-06-19 00:39:14,797 : List Bundle Creation Logger_3_m : Getting list bundle for MAN EU/1/15/1046/006  | H | CAP |  el | 3 | Elocta_clean_ ΦΥΛΛΟ ΟΔΗΓΙΩΝ ΧΡΗΣΗΣ.json
2021-06-19 00:39:15,654 : List Bundle Creation Logger_3_m : Getting list bundle for MAN EU/1/15/1046/007  | H | CAP |  el | 3 | Elocta_clean_ ΦΥΛΛΟ ΟΔΗΓΙΩΝ ΧΡΗΣ

b'<List xmlns="http://hl7.org/fhir"><id value="5829d349-155f-4f0a-ac97-6a29dd532996" /><identifier><system value="http://spor.ema.europa.eu/v2/medicine-name" /><value value="elocta" /></identifier><identifier><system value="http://spor.ema.europa.eu/v2/marketing-authorisation-numbers" /><value value="EU/1/15/1046/001" /></identifier><identifier><system value="http://spor.ema.europa.eu/v2/marketing-authorisation-numbers" /><value value="EU/1/15/1046/002" /></identifier><identifier><system value="http://spor.ema.europa.eu/v2/marketing-authorisation-numbers" /><value value="EU/1/15/1046/003" /></identifier><identifier><system value="http://spor.ema.europa.eu/v2/marketing-authorisation-numbers" /><value value="EU/1/15/1046/004" /></identifier><identifier><system value="http://spor.ema.europa.eu/v2/marketing-authorisation-numbers" /><value value="EU/1/15/1046/005" /></identifier><identifier><system value="http://spor.ema.europa.eu/v2/marketing-authorisation-numbers" /><value value="EU/1/15/

2021-06-19 00:39:27,453 : List Bundle Creation Logger_3_m : List update successfully completed 5829d349-155f-4f0a-ac97-6a29dd532996 | H | CAP |  el | 3 | Elocta_clean_ ΦΥΛΛΟ ΟΔΗΓΙΩΝ ΧΡΗΣΗΣ.json
2021-06-19 00:39:27,458 : Flow Logger HTML_o : Completed list bundle update/addition | H | CAP |  el | HTML | Elocta_clean.htm
2021-06-19 00:39:27,461 : Flow Logger HTML_o : 3: Update/Add List Bundle,0.2856 Min,0.340737 MB,0.654371 MB,54.3%
 | H | CAP |  el | HTML | Elocta_clean.htm
2021-06-19 00:39:27,464 : Flow Logger HTML_o : Completed Processing Partitioned Jsons | H | CAP |  el | HTML | Elocta_clean.htm
2021-06-19 00:39:27,469 : Flow Logger HTML_o : 3: Completed,0.0001 Min,0.150088 MB,0.152428 MB,54.3%
 | H | CAP |  el | HTML | Elocta_clean.htm
2021-06-19 00:39:27,472 : Flow Logger HTML_o : Final Metrics,13.2269 Min,0.0 MB,33.136526 MB,54.3%
 | H | CAP |  el | HTML | Elocta_clean.htm


Metrics : 3: Update/Add List Bundle,0.2856 Min,0.340737 MB,0.654371 MB,54.3%

Metrics : 3: Completed,0.0001 Min,0.150088 MB,0.152428 MB,54.3%

Metrics : Final Metrics,13.2269 Min,0.0 MB,33.136526 MB,54.3%



In [ ]:

inputList = [
            'GONAL-f~H~CAP~el~2021-06-06T10-02-52Z.zip',
             'Ovaleap~H~CAP~el~2021-06-07T09-07-39Z.zip',
            'Karvea~H~CAP~el~2021-05-21T11-44-12Z.zip']
runAllTest(inputList)


GONAL-f~H~CAP~el~2021-06-06T10-02-52Z.zip F:\Projects\EMA\Repository\EMA EPI PoC\function_code\inputblob F:\Projects\EMA\Repository\EMA EPI PoC\function_code\work\H\CAP\GONAL-f\el\2021-06-06T10-02-52Z F:\Projects\EMA\Repository\EMA EPI PoC\function_code\control
Already Present


2021-06-19 01:17:07,723 : Flow Logger HTML_5 : Starting HTML Conversion To Json | H | CAP |  el | HTML | gonal-f_clean.htm
2021-06-19 01:17:07,737 : Style Dictionary_a : Reading style dictionary in file: rule_dictionary_el.json | H | CAP |  el | HTML | gonal-f_clean.htm
2021-06-19 01:17:07,737 : Style Dictionary_a : Reading style dictionary in file: rule_dictionary_el.json | H | CAP |  el | HTML | gonal-f_clean.htm
2021-06-19 01:17:07,854 : Style Dictionary_a : Qrd Section Keys Retrieved For Style Dictionary: ΠΕΡΙΛΗΨΗ ΤΩΝ ΧΑΡΑΚΤΗΡΙΣΤΙΚΩΝ ΤΟΥ ΠΡΟΪΟΝΤΟΣ, ΠΑΡΑΡΤΗΜΑ II, A. ΕΠΙΣΗΜΑΝΣΗ, B. ΦΥΛΛΟ ΟΔΗΓΙΩΝ ΧΡΗΣΗΣ | H | CAP |  el | HTML | gonal-f_clean.htm
2021-06-19 01:17:07,854 : Style Dictionary_a : Qrd Section Keys Retrieved For Style Dictionary: ΠΕΡΙΛΗΨΗ ΤΩΝ ΧΑΡΑΚΤΗΡΙΣΤΙΚΩΝ ΤΟΥ ΠΡΟΪΟΝΤΟΣ, ΠΑΡΑΡΤΗΜΑ II, A. ΕΠΙΣΗΜΑΝΣΗ, B. ΦΥΛΛΟ ΟΔΗΓΙΩΝ ΧΡΗΣΗΣ | H | CAP |  el | HTML | gonal-f_clean.htm


gonal-f_clean.htm
['F:', 'Projects', 'EMA', 'Repository', 'EMA EPI PoC', 'function_code', 'work', 'H', 'CAP', 'GONAL-f', 'el', '2021-06-06T10-02-52Z'] gonal-f_clean.htm
2021-06-06T10-02-52Z el GONAL-f CAP H
------------- F:\Projects\EMA\Repository\EMA EPI PoC\function_code\work\H\CAP\GONAL-f\el\2021-06-06T10-02-52Z\outputJSON\gonal-f_clean.txt -----------------
F:\Projects\EMA\Repository\EMA EPI PoC\function_code\work\H\CAP\GONAL-f\el\2021-06-06T10-02-52Z\gonal-f_clean.htm F:\Projects\EMA\Repository\EMA EPI PoC\function_code\work\H\CAP\GONAL-f\el\2021-06-06T10-02-52Z\outputJSON\gonal-f_clean.json


2021-06-19 01:17:27,226 : Parser_O : Style Information Stored In File: F:\Projects\EMA\Repository\EMA EPI PoC\function_code\work\H\CAP\GONAL-f\el\2021-06-06T10-02-52Z\outputJSON\gonal-f_clean.txt | H | CAP |  el | HTML | gonal-f_clean.htm
2021-06-19 01:17:27,226 : Parser_O : Style Information Stored In File: F:\Projects\EMA\Repository\EMA EPI PoC\function_code\work\H\CAP\GONAL-f\el\2021-06-06T10-02-52Z\outputJSON\gonal-f_clean.txt | H | CAP |  el | HTML | gonal-f_clean.htm


!!!!!!!!!!! IN TABLE !!!!!!!!!!!!!!!!!
!!!!!!!!!!! IN TABLE !!!!!!!!!!!!!!!!!
!!!!!!!!!!! IN TABLE !!!!!!!!!!!!!!!!!
!!!!!!!!!!! IN TABLE !!!!!!!!!!!!!!!!!
!!!!!!!!!!! IN TABLE !!!!!!!!!!!!!!!!!


...

2021-06-19 01:43:55,127 : Heading Extraction gonal-f_clean_ ΦΥΛΛΟ ΟΔΗΓΙΩΝ ΧΡΗΣΗΣ.json_V : Match Passed : <=4|4.17|(96, 96, 100)|0.99| | H | CAP |  el | 3 | gonal-f_clean_ ΦΥΛΛΟ ΟΔΗΓΙΩΝ ΧΡΗΣΗΣ.json | Doc txt :- 'Άλλα φάρμακα και GONAL‑f' | Qrd txt :- 'Άλλα φάρμακα και GONAL-f' | Matched :- 'True'
2021-06-19 01:43:55,140 : Heading Extraction gonal-f_clean_ ΦΥΛΛΟ ΟΔΗΓΙΩΝ ΧΡΗΣΗΣ.json_V : Validation Flow Is Broken | H | CAP |  el | 3 | gonal-f_clean_ ΦΥΛΛΟ ΟΔΗΓΙΩΝ ΧΡΗΣΗΣ.json | currHeadId :- '23009' | prevHeadingCurrId :- '' | prevHeadingFoundId :- '23007'
2021-06-19 01:43:55,153 : Heading Extraction gonal-f_clean_ ΦΥΛΛΟ ΟΔΗΓΙΩΝ ΧΡΗΣΗΣ.json_V : Validation Passed | H | CAP |  el | 3 | gonal-f_clean_ ΦΥΛΛΟ ΟΔΗΓΙΩΝ ΧΡΗΣΗΣ.json | currHeadId :- '23009' | prevHeadingCurrId :- '23007' | prevHeadingFoundId :- '23007'
2021-06-19 01:44:01,011 : Heading Extraction gonal-f_clean_ ΦΥΛΛΟ ΟΔΗΓΙΩΝ ΧΡΗΣΗΣ.json_V : Match Passed : SpecialCase3|122.22|(62, 72, 86)|0.89| | H | CAP |  el | 3 | gonal-f_clean_ ΦΥΛ

2021-06-19 01:47:05,856 : Heading Extraction gonal-f_clean_ ΦΥΛΛΟ ΟΔΗΓΙΩΝ ΧΡΗΣΗΣ.json_V : Match Passed : Contains<>|42.86|(67, 69, 78)|0.845| | H | CAP |  el | 3 | gonal-f_clean_ ΦΥΛΛΟ ΟΔΗΓΙΩΝ ΧΡΗΣΗΣ.json | Doc txt :- 'Εάν ξεχάσετε να χρησιμοποιήσετε το GONAL‑f' | Qrd txt :- '3. Πώς να <πάρετε> <χρησιμοποιήσετε> το GONAL-f' | Matched :- 'True'
2021-06-19 01:47:05,897 : Heading Extraction gonal-f_clean_ ΦΥΛΛΟ ΟΔΗΓΙΩΝ ΧΡΗΣΗΣ.json_V : Validation Failed Package Leaflet Special Case L1 cannot repeat after itself. | H | CAP |  el | 3 | gonal-f_clean_ ΦΥΛΛΟ ΟΔΗΓΙΩΝ ΧΡΗΣΗΣ.json | currHeadId :- '23014' | prevHeadingCurrId :- '' | prevHeadingFoundId :- '23016'
2021-06-19 01:47:06,237 : Heading Extraction gonal-f_clean_ ΦΥΛΛΟ ΟΔΗΓΙΩΝ ΧΡΗΣΗΣ.json_V : Match Passed : Contains<>|28.57|(86, 74, 95)|0.947| | H | CAP |  el | 3 | gonal-f_clean_ ΦΥΛΛΟ ΟΔΗΓΙΩΝ ΧΡΗΣΗΣ.json | Doc txt :- 'Εάν ξεχάσετε να χρησιμοποιήσετε το GONAL‑f' | Qrd txt :- 'Εάν ξεχάσετε να <πάρετε> <χρησιμοποιήσετε> το GONAL-f' | Matched

2021-06-19 01:53:08,293 : Heading Extraction gonal-f_clean_ ΦΥΛΛΟ ΟΔΗΓΙΩΝ ΧΡΗΣΗΣ.json_V : Validation Passed | H | CAP |  el | 3 | gonal-f_clean_ ΦΥΛΛΟ ΟΔΗΓΙΩΝ ΧΡΗΣΗΣ.json | currHeadId :- '23026' | prevHeadingCurrId :- '23025' | prevHeadingFoundId :- '23025'



OriginalCheck



2021-06-19 01:53:20,337 : Heading Extraction gonal-f_clean_ ΦΥΛΛΟ ΟΔΗΓΙΩΝ ΧΡΗΣΗΣ.json_V : Match Passed : SpecialCase1|22.08|(90, 99, 95)|0.98| | H | CAP |  el | 3 | gonal-f_clean_ ΦΥΛΛΟ ΟΔΗΓΙΩΝ ΧΡΗΣΗΣ.json | Doc txt :- 'Το παρόν φύλλο οδηγιών χρήσης αναθεωρήθηκε για τελευταία φορά στις {ΜΜ/ΕΕΕΕ}.' | Qrd txt :- 'Το παρόν φύλλο οδηγιών χρήσης αναθεωρήθηκε για τελευταία φορά στις <{ΜΜ/ΕΕΕΕ}> <{μήνας ΕΕΕΕ}.>' | Matched :- 'True'
2021-06-19 01:53:20,356 : Heading Extraction gonal-f_clean_ ΦΥΛΛΟ ΟΔΗΓΙΩΝ ΧΡΗΣΗΣ.json_V : Validation Flow Is Broken | H | CAP |  el | 3 | gonal-f_clean_ ΦΥΛΛΟ ΟΔΗΓΙΩΝ ΧΡΗΣΗΣ.json | currHeadId :- '23028' | prevHeadingCurrId :- '' | prevHeadingFoundId :- '23026'
2021-06-19 01:53:20,372 : Heading Extraction gonal-f_clean_ ΦΥΛΛΟ ΟΔΗΓΙΩΝ ΧΡΗΣΗΣ.json_V : Validation Passed | H | CAP |  el | 3 | gonal-f_clean_ ΦΥΛΛΟ ΟΔΗΓΙΩΝ ΧΡΗΣΗΣ.json | currHeadId :- '23028' | prevHeadingCurrId :- '23026' | prevHeadingFoundId :- '23026'
